In [1]:
import sys
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import ariel_load
import tqdm
import copy
import importlib
import ariel_diagnostics
train_data = kgs.load_all_train_data()
kgs.sanity_checks_active = False

local


In [ ]:
# FGS = []
# AIRS = []
# N_FGS=2500
# N_AIRS=500
# loaders = ariel_load.default_loaders()
# for d in tqdm.tqdm(train_data):
#     for t in d.transits:
#         tt = copy.deepcopy(t)
#         tt.load_to_step(2, d, loaders)
#         for ii in range(tt.data[0].data.shape[0]//N_FGS):
#             FGS.append(cp.mean(tt.data[0].data[N_FGS*ii:(N_FGS)*(ii+1),...],0).get())
#         for ii in range(tt.data[1].data.shape[0]//N_AIRS):
#             AIRS.append(cp.mean(tt.data[1].data[N_AIRS*ii:(N_AIRS)*(ii+1),...],0).get())
# del tt
# FGS = cp.array(FGS)
# AIRS = cp.array(AIRS)
# data = [FGS, AIRS]
# kgs.dill_save(kgs.temp_dir + '/explore_sensor_modes.pickle', data)
data = kgs.dill_load(kgs.temp_dir + '/explore_sensor_modes.pickle')
print([d.shape for d in data])

 11%|█████████                                                                       | 124/1100 [02:50<29:11,  1.79s/it]

In [ ]:
sens_id = 0
plt.figure()
plt.imshow(cp.log(cp.nanmean(data[sens_id],0)).get()/np.log(10))
plt.colorbar()
plt.figure()
plt.plot(cp.nanmean(data[sens_id],axis=(1,2)).get())

In [ ]:
# def nan_pca(mat, n_components):
#     _, *rest = mat.shape
#     print(rest)
#     mat = copy.deepcopy(mat)
#     mat = mat.reshape((mat.shape[0],-1))
#     mat0 = copy.deepcopy(mat)
#     mat0[:,cp.any(cp.isnan(mat0),0)] = 0
#     u,s,components = cp.linalg.svd(mat0, full_matrices=False, compute_uv=True)
#     weights = u*s  
#     components = components[:n_components,:]
#     weights = weights[:,:n_components]
#     return weights, components

# def nan_pca(mat, n_components, max_iter=50, tol=1e-6, verbose=False):
#     """
#     Perform PCA on data with missing values using an EM-like iterative approach.

#     Args:
#         mat (cp.ndarray): Input data, shape (n_samples, ...). May contain NaNs.
#         n_components (int): Number of principal components to retain.
#         max_iter (int): Maximum number of iterations.
#         tol (float): Convergence tolerance on reconstruction error change.
#         verbose (bool): If True, prints iteration diagnostics.

#     Returns:
#         W (cp.ndarray): Projected scores (n_samples, n_components).
#         C (cp.ndarray): Principal components (n_components, n_features).
#         S (cp.ndarray): Singular values for each component (n_components,).
#     """
#     # Flatten per-sample features
#     orig_shape = mat.shape
#     X = mat.copy()
#     n_samples = X.shape[0]
#     X = X.reshape(n_samples, -1)
#     mask = ~cp.isnan(X)

#     # Initialize missing entries to zero
#     X_filled = X.copy()
#     X_filled[~mask] = 0

#     # Initial SVD
#     U, S_full, Vt = cp.linalg.svd(X_filled, full_matrices=False)
#     # Truncate to n_components
#     S = S_full[:n_components]
#     C = Vt[:n_components, :]
#     W = U[:, :n_components] * S

#     prev_error = cp.inf
#     for itr in range(1, max_iter+1):
#         # E-step: fill missing values with current reconstruction
#         X_hat = cp.dot(W, C)
#         X_filled[~mask] = X_hat[~mask]

#         # M-step: recompute PCA on filled data
#         U, S_full, Vt = cp.linalg.svd(X_filled, full_matrices=False)
#         S = S_full[:n_components]
#         C = Vt[:n_components, :]
#         W = U[:, :n_components] * S

#         # Compute reconstruction error on observed entries
#         X_hat = cp.dot(W, C)
#         err = cp.linalg.norm((X - X_hat)[mask])
#         if verbose:
#             print(f"Iteration {itr}: error = {err:.6f}")

#         # Check convergence
#         if abs(prev_error - err) < tol:
#             if verbose:
#                 print(f"Converged at iteration {itr} with error {err:.6f}")
#             break
#         prev_error = err

#     new_shape = list(orig_shape)
#     new_shape[0] = C.shape[0]
#     C = C.reshape(new_shape)
#     return W, C, S

# #for ii in range(5):
# #   weights,components, S = nan_pca(data[sens_id], ii, tol=0.1, verbose=False)
# #   print( cp.sqrt(cp.nanmean( (data[sens_id]-cp.tensordot(weights,components,axes=([1], [0])))**2)))
# weights,components, S = nan_pca(data[sens_id], 25, tol=0.1, verbose=True)
# print( cp.sqrt(cp.nanmean( (data[sens_id]-cp.tensordot(weights,components,axes=([1], [0])))**2)))
# plt.figure();
# plt.semilogy(S.get());

In [ ]:
def nan_pca(mat, n_components, max_iter=50, tol=1e-6, verbose=False):
    """
    Perform PCA on data with missing values using an alternating least squares approach,
    fitting directly on observed entries (unfilled data) and initializing with SVD.

    Args:
        mat (cp.ndarray): Input data, shape (n_samples, ...). May contain NaNs.
        n_components (int): Number of principal components to retain.
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance on reconstruction error change.
        verbose (bool): If True, prints iteration diagnostics.

    Returns:
        W (cp.ndarray): Projected scores (n_samples, n_components).
        C (cp.ndarray): Principal components (n_components, n_features).
    """
    # Flatten per-sample features
    X = cp.array(mat, copy=True)
    n_samples = X.shape[0]
    X = X.reshape(n_samples, -1)
    mask = ~cp.isnan(X)
    n_features = X.shape[1]

    # Initialize missing entries to zero for initial SVD
    X_mean = X.copy()
    X_mean[...] = cp.nanmean(X,0)[None,:]
    X_filled = X.copy()
    print(X_filled.shape)
    X_filled[~mask] = X_mean[~mask]
    print(X_filled.shape)
    
    #X_filled = mat.copy()
    #ariel_load.inpaint_along_axis_inplace(X_filled,2)
    #X_filled = X_filled.reshape(n_samples,-1)

    # Initial SVD on filled data to get starting W and C
    U, S_full, Vt = cp.linalg.svd(X_filled, full_matrices=False)
    W = U[:, :n_components] * S_full[:n_components]
    C = Vt[:n_components, :]

    prev_error = cp.inf
    # Precompute observed indices
    obs_indices = [cp.where(mask[i])[0] for i in range(n_samples)]
    feature_indices = [cp.where(mask[:, j])[0] for j in range(n_features)]

    for itr in range(1, max_iter + 1):
        # Update W: per-sample direct least squares on observed entries of X
        for i in range(n_samples):
            idx = obs_indices[i]
            Ci = C[:, idx]  # (n_components, n_obs)
            xi = X[i, idx]  # (n_obs,)
            # w_i = (Ci Ci^T)^{-1} Ci xi
            W[i] = cp.linalg.solve(Ci @ Ci.T, Ci @ xi)

        # Update C: per-feature direct least squares on observed entries of X
        for j in range(n_features):
            idx = feature_indices[j]
            Wj = W[idx]     # (n_obs, n_components)
            xj = X[idx, j]  # (n_obs,)
            # c_j = (Wj^T Wj)^{-1} Wj^T xj
            C[:, j] = cp.linalg.solve(Wj.T @ Wj, Wj.T @ xj)

        # Compute reconstruction error on observed entries
        X_hat = W @ C
        err = cp.linalg.norm((X - X_hat)[mask])
        if verbose:
            print(f"Iteration {itr}: error = {err:.6f}")
        if abs(prev_error - err) < tol:
            if verbose:
                print(f"Converged at iteration {itr} with error {err:.6f}")
            break
        prev_error = err

    new_shape = list(mat.shape)
    new_shape[0] = C.shape[0]
    C = C.reshape(new_shape)
    return W, C

import cupyx.scipy.sparse.linalg as cusparse
def nan_pca(mat, n_components, max_iter=50, tol=1e-6, verbose=False):
    """
    Perform PCA on data with missing values using an alternating least squares approach,
    fitting directly on observed entries (unfilled data) with fully GPU-parallel batch solves.

    Args:
        mat (cp.ndarray): Input data, shape (n_samples, ...). May contain NaNs.
        n_components (int): Number of principal components to retain.
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance on reconstruction error change.
        verbose (bool): If True, prints iteration diagnostics.

    Returns:
        W (cp.ndarray): Projected scores (n_samples, n_components).
        C (cp.ndarray): Principal components (n_components, n_features).
    """
    # Flatten per-sample features
    X = cp.array(mat, copy=True)
    n_samples = X.shape[0]
    X = X.reshape(n_samples, -1)
    mask = (~cp.isnan(X))
    n_features = X.shape[1]
    #print(cp.sum(cp.sum(mask,1)<n_components))
    #print(cp.sum(cp.sum(mask,0)<n_components))
    X[cp.sum(mask,1)<n_components+1,:] = 0.
    X[:,cp.sum(mask,0)<n_components+1] = 0.
    mask = (~cp.isnan(X))
    #plt.figure()
    #plt.plot(cp.sum(mask,0).get())
    X[~mask] = 0.
    #X[...] = 0.
    
    #print(cp.min(cp.sum(~mask,0)), cp.min(cp.sum(~mask,1)))
    
    

#     # Initialize missing entries to zero for initial SVD
#     X_mean = X.copy()
#     X_mean[...] = cp.nanmean(X,0)[None,:]
#     X_filled = X.copy()
#     print(X_filled.shape)
#     X_filled[~mask] = X_mean[~mask]
#     print(X_filled.shape)
#     assert not cp.any(cp.isnan(X_filled))
#     #print(X.shape)
#     #X_filled = copy.deepcopy(mat)
#     #ariel_load.inpaint_along_axis_inplace(X_filled, axis=2)
#     #X_filled = X_filled.reshape(n_samples, -1)

#     # Initial SVD on filled data
#     # Initial SVD on filled data (optionally partial)
#     # compute top-k singular values/vectors via sparse solver
#     U_k, S_k, Vt_k = cusparse.svds(X_filled, k=n_components)
#     # svds returns ascending S; reorder to descending
#     idx = cp.argsort(S_k)[::-1]
#     S = S_k[idx]
#     U = U_k[:, idx]
#     Vt = Vt_k[idx, :]
    
 
#     # else:
#     #     U_full, S_full, Vt_full = cp.linalg.svd(X_filled, full_matrices=False)
#     #     S = S_full[:n_components]
#     #     U = U_full[:, :n_components]
#     #     Vt = Vt_full[:n_components, :]
    
#     # Initialize W and C
#     W = U * S
#     C = Vt
    
    rng = cp.random.default_rng(seed=42)
    W = rng.standard_normal((n_samples, n_components), dtype=X.dtype)
    C = rng.standard_normal((n_components, n_features), dtype=X.dtype)
    
    mask = mask.astype(X.dtype)  # float mask for weighting

    prev_error = cp.inf
    for itr in range(1, max_iter + 1):
        # === Batch-solve for C: for each j, (W^T M^j W) c_j = W^T (M^j x^j) ===
        # Here M^j is mask[:, j] for feature j
        # Compute A2_batch[j,p,q] = sum_i mask[i,j] * W[i,p] * W[i,q]
        A2_batch = cp.einsum('ip,ij,iq->jpq', W, mask, W)
        # Compute B2_batch[j,p] = sum_i mask[i,j] * W[i,p] * X[i,j]
        B2_batch = cp.einsum('ip,ij,ij->jp', W, mask, X)
        # Solve for C: shape (n_components, n_features)
        C = cp.linalg.solve(A2_batch, B2_batch[..., None]).squeeze(-1).T
        del A2_batch, B2_batch
        C= cp.linalg.qr(C.T)[0].T
        #C = C.T
        #print(C@C.T)
        
        # === Batch-solve for W: for each i, (C M_i C^T) w_i = C (M_i x_i) ===
        # Compute A_batch[i,a,c] = sum_j mask[i,j] * C[a,j] * C[c,j]
        A_batch = cp.einsum('aj,ij,cj->iac', C, mask, C)
        # Compute B_batch[i,a] = sum_j mask[i,j] * C[a,j] * X[i,j]
        B_batch = cp.einsum('aj,ij,ij->ia', C, mask, X)
        
        #print(cp.sum(cp.isnan(A_batch)), cp.sum(cp.isnan(B_batch)))
        # Solve for W: shape (n_samples, n_components)
        # cp.linalg.solve supports batch: A_batch @ W_batch[...,None] = B_batch[...,None]
        W = cp.linalg.solve(A_batch, B_batch[..., None]).squeeze(-1)
        del A_batch, B_batch
        
        

        
        
              
        

        # Compute reconstruction and error
        err = cp.linalg.norm((X - W @ C) * mask)
        if verbose:
            print(f"Iteration {itr}: error = {err:.6f}")
        if abs(prev_error - err) < tol:
            if verbose:
                print(f"Converged at iteration {itr} with error {err:.6f}")
            break
        prev_error = err

    new_shape = list(mat.shape)
    new_shape[0] = C.shape[0]
    C = C.reshape(new_shape)
    S_post = cp.linalg.norm(W, axis=0)
    order = cp.argsort(S_post)[::-1]
    W = W[:, order]
    C = C[order,:]
    S_post = S_post[order]
    return W, C, S_post



#    weights,components, S = nan_pca(data[sens_id], ii, tol=0.1, verbose=False)
#    print( cp.sqrt(cp.nanmean( (data[sens_id]-cp.tensordot(weights,components,axes=([1], [0])))**2)))
weights,components,S = nan_pca(data[sens_id], 25, tol=1e-1, verbose=True)
plt.figure();plt.semilogy(S.get())

In [ ]:
print( cp.sqrt(cp.nanmean( (data[sens_id]-cp.tensordot(weights,components,axes=([1], [0])))**2)))
print( cp.sqrt(cp.nanmean( (data[sens_id]-cp.tensordot(weights,components,axes=([1], [0])))**2)))
for ii in range(components.shape[0]):
    plt.figure()
    plt.imshow(components[ii,...].get(), aspect='auto')
    plt.colorbar()
    plt.title(kgs.rms(weights[:,ii]).get())

In [7]:
print(components.shape)

(10, 32, 32)


In [8]:
weights.shape

(2916, 10)